In [85]:
import os
import random
import numpy as np
import pandas as pd

from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GATConv


# ============================================================
# 1) Reproducibility
# ============================================================
def set_seed(seed: int = 2):
    """Set seeds for python/numpy/torch to improve reproducibility."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


# ============================================================
# 2) Feature config and edge-type IDs
# ============================================================
FEATURE_COLS = [
    "NDVI", "Disroad", "Driver", "Landuse",
    "TWI", "SPI", "Relief", "Curvature",
    "aspect", "Slope", "Lil", "Elevation", "rainfall"
]

# Optional: type-aware feature subsets used for within-type similarity among positive nodes
TYPE_FEATURES = {
    1: ["Driver", "TWI", "rainfall", "Elevation", "NDVI", "Slope"],
    2: ["Disroad", "Landuse", "NDVI"],
    3: ["Slope", "Relief", "TWI"],
    4: ["TWI", "Relief", "Slope", "SPI", "NDVI"],
    5: ["Driver", "NDVI", "Elevation", "TWI", "Slope"],
}

# Edge types (must remain in [0..4])
E_LL = 0   # landslide-landslide (within-type positives)
E_NN = 1   # nonlandslide-nonlandslide (negatives)
E_LN = 2   # landslide-nonlandslide (bidirectional contrast edges)
E_UL = 3   # unlabeled pseudo-positive <-> train positive anchors (dynamic)
E_UN = 4   # unlabeled pseudo-negative <-> train negative anchors (dynamic)


# ============================================================
# 3) Optional synthetic dataset (for runnable demo without real data)
# ============================================================
def generate_synthetic_df(
    n_total: int = 2000,
    n_pos: int = 300,
    n_neg: int = 300,
    seed: int = 2
) -> pd.DataFrame:
    """
    Generate a synthetic dataset for sanity checks.
    Columns:
      - Id: int
      - label: 1/0/-1 (1=positive, 0=negative, -1=unlabeled)
      - ls_type: 1..5 for positives, 0 otherwise
      - FEATURE_COLS: float in [0,1]
    """
    assert n_pos + n_neg <= n_total
    set_seed(seed)

    n_unl = n_total - n_pos - n_neg
    pos_types = np.random.choice([1, 2, 3, 4, 5], size=n_pos, replace=True)

    base_neg = np.array([0.45] * len(FEATURE_COLS), dtype=float)
    base_pos = np.array([0.55] * len(FEATURE_COLS), dtype=float)

    type_shift = {t: np.zeros(len(FEATURE_COLS), dtype=float) for t in [1, 2, 3, 4, 5]}
    col_to_i = {c: i for i, c in enumerate(FEATURE_COLS)}

    for t, cols in TYPE_FEATURES.items():
        for c in cols:
            type_shift[t][col_to_i[c]] += 0.18

    def sample_block(n, mean_vec, noise=0.10):
        X = mean_vec + noise * np.random.randn(n, len(FEATURE_COLS))
        return np.clip(X, 0.0, 1.0)

    X_pos = np.zeros((n_pos, len(FEATURE_COLS)), dtype=float)
    for t in [1, 2, 3, 4, 5]:
        idx = np.where(pos_types == t)[0]
        if idx.size:
            X_pos[idx] = sample_block(idx.size, base_pos + type_shift[t], noise=0.08)

    X_neg = sample_block(n_neg, base_neg, noise=0.10)

    unl_mix = np.random.rand(n_unl)
    X_unl = np.zeros((n_unl, len(FEATURE_COLS)), dtype=float)
    for i in range(n_unl):
        if unl_mix[i] < 0.45:
            X_unl[i] = sample_block(1, base_neg, noise=0.12)[0]
        else:
            t = np.random.choice([1, 2, 3, 4, 5])
            X_unl[i] = sample_block(1, base_pos + 0.7 * type_shift[t], noise=0.10)[0]

    Id = np.arange(1, n_total + 1, dtype=int)
    label = np.array([1] * n_pos + [0] * n_neg + [-1] * n_unl, dtype=int)
    ls_type = np.array(list(pos_types) + [0] * n_neg + [0] * n_unl, dtype=int)

    X = np.vstack([X_pos, X_neg, X_unl])
    df = pd.DataFrame(X, columns=FEATURE_COLS)
    df.insert(0, "ls_type", ls_type)
    df.insert(0, "label", label)
    df.insert(0, "Id", Id)
    return df


# ============================================================
# 4) Graph utilities: cosine top-k edges
# ============================================================
def topk_cosine_edges(global_indices: np.ndarray, X: np.ndarray, k: int):
    """
    Build within-set top-k cosine similarity edges.
    global_indices are node indices in the full graph (aligned with df row indices).
    """
    n = X.shape[0]
    if n == 0:
        return []
    Xn = normalize(X, axis=1)
    S = Xn @ Xn.T  # (n, n)

    edges = []
    for i in range(n):
        order = np.argsort(-S[i])
        top = order[: min(k, n)]  # may include self-loop
        src = int(global_indices[i])
        for j in top:
            edges.append((src, int(global_indices[j])))
    return edges


def topk_cosine_cross_edges(src_global, X_src, dst_global, X_dst, k: int, bidirectional: bool = True):
    """
    Build cross-set edges using top-k cosine similarity from src -> dst.
    If bidirectional=True, add reverse edges as well.
    """
    if X_src.shape[0] == 0 or X_dst.shape[0] == 0:
        return []

    A = normalize(X_src, axis=1)
    B = normalize(X_dst, axis=1)
    S = A @ B.T  # (nsrc, ndst)

    edges = []
    for i in range(S.shape[0]):
        order = np.argsort(-S[i])
        top = order[: min(k, S.shape[1])]
        s = int(src_global[i])
        for j in top:
            t = int(dst_global[j])
            edges.append((s, t))
            if bidirectional:
                edges.append((t, s))
    return edges


# ============================================================
# 5) Build static graph (E_LL / E_NN / E_LN)
# ============================================================
def build_static_graph(df: pd.DataFrame, eta_intra: int = 7, eta_nn: int = 7, eta_ln: int = 7):
    """
    Static graph uses only labeled nodes (label in {1,0}):
      - E_LL: within-type edges among positive clusters (ls_type in 1..5)
      - E_NN: within-class edges among negatives
      - E_LN: bidirectional cross-class edges (positive <-> negative)

    Note:
      Unlabeled nodes (label=-1) are not connected by static edges here.
      They receive messages only when dynamic edges (E_UL/E_UN) are added during training.
    """
    node_ids = df["Id"].values
    pos_idx = df.index[df["label"] == 1].to_numpy()
    neg_idx = df.index[df["label"] == 0].to_numpy()

    edge_list = []
    edge_type = []

    # E_LL: within-type similarity among positive nodes
    for t in [1, 2, 3, 4, 5]:
        sub = df[(df["label"] == 1) & (df["ls_type"] == t)]
        if sub.shape[0] == 0:
            continue

        cols = TYPE_FEATURES.get(t, FEATURE_COLS)
        if not all(c in sub.columns for c in cols):
            cols = FEATURE_COLS

        gidx = sub.index.to_numpy()
        X = sub[cols].values.astype(float)
        edges = topk_cosine_edges(gidx, X, eta_intra)
        edge_list.extend(edges)
        edge_type.extend([E_LL] * len(edges))

    # E_NN: within-class similarity among negative nodes
    if neg_idx.size > 0:
        Xn = df.loc[neg_idx, FEATURE_COLS].values.astype(float)
        edges_nn = topk_cosine_edges(neg_idx, Xn, eta_nn)
        edge_list.extend(edges_nn)
        edge_type.extend([E_NN] * len(edges_nn))

    # E_LN: cross-class edges (bidirectional)
    if pos_idx.size > 0 and neg_idx.size > 0:
        Xp = df.loc[pos_idx, FEATURE_COLS].values.astype(float)
        Xn = df.loc[neg_idx, FEATURE_COLS].values.astype(float)
        edges_ln = topk_cosine_cross_edges(pos_idx, Xp, neg_idx, Xn, eta_ln, bidirectional=True)
        edge_list.extend(edges_ln)
        edge_type.extend([E_LN] * len(edges_ln))

    edge_index = torch.tensor(edge_list, dtype=torch.long).t().contiguous()
    edge_type = torch.tensor(edge_type, dtype=torch.long)
    return node_ids, edge_index, edge_type


# ============================================================
# 6) Train/Val/Test split (labeled nodes only)
# ============================================================
def make_splits(
    y: torch.Tensor,
    seed: int = 2,
    train_ratio: float = 0.7,
    val_ratio: float = 0.15,
    test_ratio: float = 0.15
):
    """
    Split labeled nodes only (y != -1):
      1) labeled -> train/test
      2) train -> train/val
    """
    assert abs(train_ratio + val_ratio + test_ratio - 1.0) < 1e-6

    labeled = (y != -1).nonzero(as_tuple=True)[0].cpu().numpy()
    y_lab = y[labeled].cpu().numpy()

    train_idx, test_idx = train_test_split(
        labeled,
        test_size=test_ratio,
        random_state=seed,
        stratify=y_lab
    )

    y_train = y[train_idx].cpu().numpy()
    val_frac_in_train = val_ratio / (train_ratio + val_ratio)

    train_idx, val_idx = train_test_split(
        train_idx,
        test_size=val_frac_in_train,
        random_state=seed,
        stratify=y_train
    )

    n = y.shape[0]
    train_mask = torch.zeros(n, dtype=torch.bool)
    val_mask = torch.zeros(n, dtype=torch.bool)
    test_mask = torch.zeros(n, dtype=torch.bool)

    train_mask[train_idx] = True
    val_mask[val_idx] = True
    test_mask[test_idx] = True
    return train_mask, val_mask, test_mask


# ============================================================
# 7) Model: node-att + edge-att (sum) -> MLP
# ============================================================
class NodeEdgeFusionGAT(nn.Module):
    """
    Two parallel branches:
      - node_att: node-level attention (no edge attributes)
      - edge_att: edge-semantic attention (edge attributes from edge types)
    Fusion: element-wise sum, then MLP classifier.
    """
    def __init__(self, in_dim, hidden_dim, num_edge_types=5, heads=5, dropout=0.3):
        super().__init__()
        self.dropout = dropout

        self.edge_type_emb = nn.Embedding(num_edge_types, in_dim)

        self.node_att = GATConv(
            in_channels=in_dim, out_channels=hidden_dim,
            heads=heads, concat=True, dropout=dropout
        )
        self.edge_att = GATConv(
            in_channels=in_dim, out_channels=hidden_dim,
            heads=heads, concat=True, dropout=dropout, edge_dim=in_dim
        )

        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim * heads, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x, edge_index, edge_type):
        edge_attr = self.edge_type_emb(edge_type)
        h_node = F.elu(self.node_att(x, edge_index))
        h_edge = F.elu(self.edge_att(x, edge_index, edge_attr))
        h = h_node + h_edge
        h = F.dropout(h, p=self.dropout, training=self.training)
        return torch.sigmoid(self.classifier(h).squeeze())


# ============================================================
# 8) Dynamic edge updater (recomputed, non-cumulative)
# ============================================================
class DynamicEdgeUpdater:
    """
    Dynamic edges are rebuilt every update (non-cumulative):
      - Anchors come from train_mask only (prevents val/test leakage).
      - Unlabeled nodes are selected by high-confidence predictions.
      - Pseudo labels are stored in data.pseudo_y but NOT used in the supervised loss.
    """
    def __init__(self, b=7, c=7, confidence_threshold=0.8):
        self.b = b
        self.c = c
        self.threshold = confidence_threshold

    def update_edges(self, data: Data, model: nn.Module):
        model.eval()
        with torch.no_grad():
            probs = model(data.x, data.edge_index, data.edge_type)

        unlabeled = (data.y == -1)
        pseudo_pos = (probs > self.threshold) & unlabeled
        pseudo_neg = (probs < (1 - self.threshold)) & unlabeled

        pos_anchor = torch.where((data.y == 1) & data.train_mask)[0]
        neg_anchor = torch.where((data.y == 0) & data.train_mask)[0]

        if pos_anchor.numel() == 0 or neg_anchor.numel() == 0:
            data.edge_index = data.edge_index_static
            data.edge_type = data.edge_type_static
            return data

        new_edges = []
        new_types = []

        # Build bidirectional edges for stable message passing
        for src in torch.where(pseudo_pos)[0]:
            sim = F.cosine_similarity(data.x[pos_anchor], data.x[src].unsqueeze(0), dim=-1)
            k = min(self.b, sim.numel())
            topk = torch.topk(sim, k=k).indices
            for dst in pos_anchor[topk]:
                s = int(src.item()); d = int(dst.item())
                new_edges.append([s, d]); new_types.append(E_UL)
                new_edges.append([d, s]); new_types.append(E_UL)

        for src in torch.where(pseudo_neg)[0]:
            sim = F.cosine_similarity(data.x[neg_anchor], data.x[src].unsqueeze(0), dim=-1)
            k = min(self.c, sim.numel())
            topk = torch.topk(sim, k=k).indices
            for dst in neg_anchor[topk]:
                s = int(src.item()); d = int(dst.item())
                new_edges.append([s, d]); new_types.append(E_UN)
                new_edges.append([d, s]); new_types.append(E_UN)

        # Rebuild dynamic graph: static edges + current dynamic edges
        if len(new_edges) > 0:
            new_edge_index = torch.tensor(new_edges, device=data.edge_index.device, dtype=torch.long).t()
            new_edge_type = torch.tensor(new_types, device=data.edge_type.device, dtype=data.edge_type.dtype)
            data.edge_index = torch.cat([data.edge_index_static, new_edge_index], dim=1)
            data.edge_type = torch.cat([data.edge_type_static, new_edge_type], dim=0)
        else:
            data.edge_index = data.edge_index_static
            data.edge_type = data.edge_type_static

        pseudo_y = torch.full_like(data.y, -1)
        pseudo_y[pseudo_pos] = 1
        pseudo_y[pseudo_neg] = 0
        data.pseudo_y = pseudo_y
        return data


# ============================================================
# 9) Evaluation helpers
# ============================================================
def eval_split(model, data, mask, criterion):
    """Compute loss/accuracy/AUC on a specific split mask."""
    model.eval()
    with torch.no_grad():
        prob = model(data.x, data.edge_index, data.edge_type)
        y_true = data.y[mask].float()
        y_prob = prob[mask]

        loss = criterion(y_prob, y_true).item()

        y_pred = (y_prob > 0.5).long().cpu().numpy()
        y_true_i = data.y[mask].long().cpu().numpy()
        acc = accuracy_score(y_true_i, y_pred)

        auc = float("nan")
        if len(np.unique(y_true_i)) == 2:
            auc = roc_auc_score(y_true_i, y_prob.detach().cpu().numpy())

    return loss, acc, auc


# ============================================================
# 10) Jupyter entry (read real data if provided, else use synthetic)
# ============================================================
set_seed(2)

# Set this to your processed file path (xlsx/csv) if available.
# Required columns: Id, label, ls_type, and FEATURE_COLS.
DATA_PATH = None

if DATA_PATH is not None and os.path.exists(DATA_PATH):
    if DATA_PATH.lower().endswith(".csv"):
        df = pd.read_csv(DATA_PATH)
    else:
        df = pd.read_excel(DATA_PATH)
else:
    df = generate_synthetic_df(n_total=2000, n_pos=300, n_neg=300, seed=2)

required_cols = ["Id", "label", "ls_type"] + FEATURE_COLS
for col in required_cols:
    if col not in df.columns:
        raise ValueError(f"Missing required column: {col}")

node_ids, edge_index, edge_type = build_static_graph(df, eta_intra=7, eta_nn=7, eta_ln=7)

x = torch.tensor(df[FEATURE_COLS].values.astype(np.float32))
y = torch.tensor(df["label"].values.astype(np.int64))

train_mask, val_mask, test_mask = make_splits(y, seed=2, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15)

data = Data(
    x=x,
    edge_index=edge_index,
    edge_type=edge_type,
    y=y,
    train_mask=train_mask,
    val_mask=val_mask,
    test_mask=test_mask
)

# Edge-type sanity check
print("edge_type min/max:", int(data.edge_type.min()), int(data.edge_type.max()))
assert int(data.edge_type.min()) >= 0
assert int(data.edge_type.max()) <= 4, "edge_type must be within 0..4."

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data = data.to(device)

model = NodeEdgeFusionGAT(
    in_dim=len(FEATURE_COLS),
    hidden_dim=128,
    num_edge_types=5,
    heads=5,
    dropout=0.3
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-8)
criterion = nn.BCELoss()

# Important: keep static backups AFTER moving data to device
data.edge_index_static = data.edge_index.clone()
data.edge_type_static = data.edge_type.clone()

updater = DynamicEdgeUpdater(b=7, c=7, confidence_threshold=0.8)

epochs = 60
update_every = 5

for epoch in range(1, epochs + 1):
    model.train()
    optimizer.zero_grad()

    # Dynamic update: recompute (non-cumulative)
    if update_every > 0 and epoch % update_every == 0:
        data = updater.update_edges(data, model)

    prob = model(data.x, data.edge_index, data.edge_type)

    # Supervised loss uses train_mask only (pseudo labels are NOT used in loss)
    loss = criterion(prob[data.train_mask], data.y[data.train_mask].float())
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0 or epoch == 1:
        tr = eval_split(model, data, data.train_mask, criterion)
        va = eval_split(model, data, data.val_mask, criterion)
        te = eval_split(model, data, data.test_mask, criterion)
        print(
            f"Epoch {epoch:03d} | "
            f"Train loss/acc/auc: {tr[0]:.4f}/{tr[1]:.4f}/{tr[2]:.4f} | "
            f"Val loss/acc/auc: {va[0]:.4f}/{va[1]:.4f}/{va[2]:.4f} | "
            f"Test loss/acc/auc: {te[0]:.4f}/{te[1]:.4f}/{te[2]:.4f}"
        )

tr = eval_split(model, data, data.train_mask, criterion)
va = eval_split(model, data, data.val_mask, criterion)
te = eval_split(model, data, data.test_mask, criterion)

print("\nFinal:")
print(f"Train: loss={tr[0]:.4f}, acc={tr[1]:.4f}, auc={tr[2]:.4f}")
print(f"Val  : loss={va[0]:.4f}, acc={va[1]:.4f}, auc={va[2]:.4f}")
print(f"Test : loss={te[0]:.4f}, acc={te[1]:.4f}, auc={te[2]:.4f}")

with torch.no_grad():
    prob_all = model(data.x, data.edge_index, data.edge_type).detach().cpu().numpy()

out_df = pd.DataFrame({"Id": node_ids, "prob": prob_all})
os.makedirs("outputs", exist_ok=True)
out_df.to_csv("outputs/predictions_all_nodes.csv", index=False)
print("Saved: outputs/predictions_all_nodes.csv")


edge_type min/max: 0 2
Epoch 001 | Train loss/acc/auc: 0.6856/0.5024/0.8347 | Val loss/acc/auc: 0.6872/0.5000/0.7827 | Test loss/acc/auc: 0.6854/0.5111/0.8474
Epoch 010 | Train loss/acc/auc: 0.5050/0.8595/0.9908 | Val loss/acc/auc: 0.5141/0.8222/0.9951 | Test loss/acc/auc: 0.4977/0.8889/0.9911
Epoch 020 | Train loss/acc/auc: 0.1648/0.9881/0.9974 | Val loss/acc/auc: 0.1834/0.9778/0.9970 | Test loss/acc/auc: 0.1800/0.9778/0.9951
Epoch 030 | Train loss/acc/auc: 0.0654/0.9905/0.9991 | Val loss/acc/auc: 0.1364/0.9667/0.9970 | Test loss/acc/auc: 0.1041/0.9778/0.9960
Epoch 040 | Train loss/acc/auc: 0.0485/0.9952/0.9991 | Val loss/acc/auc: 0.1313/0.9556/0.9960 | Test loss/acc/auc: 0.1098/0.9778/0.9936
Epoch 050 | Train loss/acc/auc: 0.0348/0.9929/1.0000 | Val loss/acc/auc: 0.2118/0.9000/0.9931 | Test loss/acc/auc: 0.1274/0.9333/0.9901
Epoch 060 | Train loss/acc/auc: 0.0430/0.9833/1.0000 | Val loss/acc/auc: 0.3883/0.8556/0.9906 | Test loss/acc/auc: 0.1883/0.9333/0.9881

Final:
Train: loss=0.043